In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 20.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 574.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [3]:
!pip install transformers torch einops wandb triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir_sm90#subdirectory=python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/vchiley/triton.git (to revision triton_pre_mlir_sm90) to /tmp/pip-install-mor5opks/triton-pre-mlir_7fbf385288f9473683c7aa3f70d8c7cd
  Running command git clone --filter=blob:none --quiet https://github.com/vchiley/triton.git /tmp/pip-install-mor5opks/triton-pre-mlir_7fbf385288f9473683c7aa3f70d8c7cd
  Running command git checkout -b triton_pre_mlir_sm90 --track origin/triton_pre_mlir_sm90
  Switched to a new branch 'triton_pre_mlir_sm90'
  Branch 'triton_pre_mlir_sm90' set up to track remote branch 'triton_pre_mlir_sm90' from 'origin'.
  Resolved https://github.com/vchiley/triton.git to commit 48b1cc9ff8b1f506ac32f2124471e2582875c008
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/bernard/mpt7b_tok")
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/bernard/mpt7b_model", quantization_config=bnb_config, device_map={"":0})

Loading /content/drive/MyDrive/bernard/mpt7b_model requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co//content/drive/MyDrive/bernard/mpt7b_model. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading /content/drive/MyDrive/bernard/mpt7b_model requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co//content/drive/MyDrive/bernard/mpt7b_model. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
You are using config.init_device='cuda:0', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [2]:
from peft import prepare_model_for_kbit_training

# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [3]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [4]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4194304 || all params: 3432255488 || trainable%: 0.12220255790002542


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [5]:
inputs = torch.load('/content/drive/MyDrive/bernard/inputs.pth')

In [6]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

dataset = TextDataset(inputs)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [7]:
# from transformers import AdamW
# import wandb
# from sklearn.metrics import classification_report
# import numpy as np

# wandb.init(project="finetuning-mpt_daevas(v1)")  # start a new run, tracking hyperparameters in 'config'

# model.train() # Set the model to training mode
# optimizer = AdamW(model.parameters(), lr= 1e-5) # Use AdamW optimizer and log the learning rate
# epochs = 15  # number of epochs, defined in the Weights & Biases interface

# for epoch in range(epochs):
#     total_loss = 0
#     predictions, true_labels = [], []

#     for batch in dataloader:
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to('cuda')
#         attention_mask = batch['attention_mask'].to('cuda')
#         outputs = model(input_ids, attention_mask=attention_mask, labels=inputs)

#         loss = outputs.loss
#         total_loss += loss.item()

#         preds = torch.argmax(outputs.logits, dim=-1)  # Get the predictions
#         predictions.extend(preds().numpy())
#         true_labels.extend(labels().numpy())

#         loss.backward()
#         optimizer.step()

#     # Compute the average loss and the classification report
#     avg_train_loss = total_loss / len(dataloader)
#     classificationRep = classification_report(true_labels, predictions, output_dict=True)

#     # Log the average training loss and classification report for this epoch to Weights & Biases
#     wandb.log({"avg_train_loss": avg_train_loss, "classification_report": classificationRep, "epoch": epoch})

# # Save the trained model as an artifact to Weights & Biases
# torch.save(model.state_dict(), "finetuning-mpt_daevas(v1).pt")
# artifact = wandb.Artifact('finetuning-mpt_daevas(v1)', type='model')
# artifact.add_file('finetuning-mpt_daevas(v1).pt')
# wandb.log_artifact(artifact)

# # Finish the Weights & Biases run
# wandb.finish()


In [ ]:
import transformers


tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        # max_steps=120,
        num_train_epochs=10,
        learning_rate=1e-5,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

wandb: Currently logged in as: knowrohit (projectsolomon). Use `wandb login --relogin` to force relogin


<ipython-input-6-73975cb8d39e>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.301900
2,2.981800
3,2.955600
4,3.795700
5,3.332400
6,3.199600
7,3.074500
8,3.713300
9,3.875600
10,3.525000


In [9]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("/content/drive/MyDrive/bernard/outputs")

In [10]:
lora_config = LoraConfig.from_pretrained('/content/drive/MyDrive/bernard/outputs')
model = get_peft_model(model, lora_config)

In [16]:
text = "what is ELECTROSTATIC POTENTIAL AND CAPACITANCE "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

what is ELECTROSTATIC POTENTIAL AND CAPACITANCE 
what is ELECTROSTATIC POTENTIAL AND CAPACITANCE
Electrostatic potential is the potential difference between two points in a charged system. It is a measure of the electric potential energy per unit charge at a point in the system. The capacitance of a capacitor is the ratio of the charge stored on the capacitor to the voltage across the capacitor. It is a measure of the ability of the capacitor to store electrical energy.#include <iostream>
using namespace std
